# Chain of Thought: Solution variants

Illustrates using Chain of Thought to fix some of the problematic queries

In [1]:
#%pip install --upgrade --quiet pydantic-ai-slim[anthropic,openai]

In [1]:
GEMINI="gemini-2.0-flash"
OPENAI="gpt-4o-mini"
CLAUDE="claude-3-7-sonnet-latest"

import os
from dotenv import load_dotenv
load_dotenv("../keys.env")
assert os.environ["GEMINI_API_KEY"][:2] == "AI",\
       "Please specify the GEMINI_API_KEY access token in keys.env file"
assert os.environ["ANTHROPIC_API_KEY"][:2] == "sk",\
       "Please specify the ANTHROPIC_API_KEY access token in keys.env file"
assert os.environ["OPENAI_API_KEY"][:2] == "sk",\
       "Please specify the OPENAI_API_KEY access token in keys.env file"

In [2]:
# Needed in Jupyter environment See: https://ai.pydantic.dev/troubleshooting/ 
import nest_asyncio
nest_asyncio.apply()

In [3]:
def zero_shot(model_id, prompt: str) -> str:
    from pydantic_ai import Agent
    agent = Agent(model_id)
    result = agent.run_sync(prompt)
    return (result.data)

## Zero-shot CoT

Simply saying "think about it step-by-step"

In [8]:
print(zero_shot(CLAUDE, """
If I have a pipe of diameter 25cm and a hose that is 100m long,
what's the flow-rate of Texas Sweet with a 7 bar pressure differential?
Think about it step-by-step.
"""))

# Flow Rate Calculation for Texas Sweet Crude Oil

To determine the flow rate of Texas Sweet crude oil through a pipe, I'll need to apply fluid dynamics principles, specifically the Darcy-Weisbach equation or Hagen-Poiseuille law.

## Given information:
- Pipe diameter (D) = 25 cm = 0.25 m
- Pipe length (L) = 100 m
- Pressure differential (ΔP) = 7 bar = 700,000 Pa
- Fluid: Texas Sweet crude oil

## Step 1: Determine the properties of Texas Sweet crude oil
Texas Sweet crude oil typically has these properties:
- Density (ρ): approximately 850 kg/m³
- Dynamic viscosity (μ): approximately 0.005-0.010 Pa·s (I'll use 0.008 Pa·s)

## Step 2: Calculate flow rate using the Hagen-Poiseuille equation
For laminar flow in a circular pipe:
Q = (π × D⁴ × ΔP) / (128 × μ × L)

Where:
- Q = volumetric flow rate (m³/s)
- π = 3.14159
- D = pipe diameter (m)
- ΔP = pressure differential (Pa)
- μ = dynamic viscosity (Pa·s)
- L = pipe length (m)

## Step 3: Substitute values and calculate
Q = (π × (0.25 m)⁴ 

## Few-shot CoT

Providing an example or two.

In [10]:
print(zero_shot(GEMINI, """
You are allowed to check in 50kg if your final destination is the United States or if you have special needs.
Otherwise, you are allowed only 40kg. 

Example 1:
Q: What's the baggage allowance for itinerary CDG-ATL-SEA?
A: The final destination, SEA, is in the United States. So, baggage allowance is 50kg.

Example 2:
Q: What's the baggage allowance for itinerary CDG-LHR-NBO?
A: The final destination, NBO, is in Kenya. So, baggage allowance is 40kg.

Q: What is the baggage allowance for the following itineary: SIN-DFW-YYZ?
"""))

A: The final destination, YYZ, is in Canada. So, baggage allowance is 40kg.



## Bridge Suit Play

let's see if CoT can help

In [23]:
print(zero_shot(GEMINI, """
You are a bridge teacher. Guide a student on how to play the given suit combination. 

Example 1:
Q: Holding AKJx opposite xxxx.
A: You have 8 cards between the two hands, and the opponents have 5 cards including the Queen.
The most likely split is 3-2. You can pick up the suit only if the Queen is onside.
Finesse the Jack.

Example 2:
Q: Holding AKJx opposite 5 small.
A: You have 9 cards between the two hands, and the opponents have 4 cards including the Queen.
The most likely split is 2-2. Therefore, play for the Queen to drop.

Q: Holding AKJxx opposite four small.
"""))

Okay, let's analyze this suit combination: **Holding AKJxx opposite four small.**

Here's how I would advise you to play this suit:

**Understanding the Situation:**

*   **You have:** AKJxx (five cards in your hand) + four small cards in the dummy = **Nine cards in the combined hands.**
*   **Opponents have:** Four cards remaining in the suit.
*   **Key Card:** You need to consider where the Queen is.

**Reasoning and Plan:**

1.  **The Most Likely Split:** With four cards outstanding, the most likely split is 2-2.

2.  **The Goal:** Since we are missing the queen, and we hold AKJ, we hope for the queen to drop.

3.  **Playing the Hand:**

    *   **Play the Ace:** Lead the Ace from your hand (or the hand with AKJxx).
    *   **Play the King:** Next, lead the King from your hand.

4.  **Possible Outcomes After Leading the Ace and King**

    *   **Queen Drops:** If the Queen appears on either the Ace or King lead, congratulations! You've captured it and can now take the rest of the tr

The line of play and the reasoning is correct.

## Where CoT won't help: data gap

CoT doesn't help if the problem is a data gap.
To solve this question, you need to give the model access to a map.
You also need a model capable of interpreting a map.
Currently (April 2025), Claude Sonnet can do this, but GPT-4o-mini cannot.

In [17]:
def answer_based_on_map(model_id, prompt: str) -> str:
    from pydantic_ai import Agent, ImageUrl
    agent = Agent(model_id)
    result = agent.run_sync([prompt, 
                             ImageUrl(url="https://upload.wikimedia.org/wikipedia/commons/thumb/7/78/India_roadway_map.svg/1639px-India_roadway_map.svg.png")
                            ])
    return (result.data)

In [18]:
print(answer_based_on_map(OPENAI,"""
If I drive 300km west of Hyderabad, where will I land up?
Think about it step-by-step.
"""))

If you drive 300 km west of Hyderabad, you can expect to land in or around a few notable areas in the state of Maharashtra. Here's a step-by-step breakdown:

1. **Starting Point**: You begin in Hyderabad, which is located in the southern part of India, in the state of Telangana.
  
2. **Direction**: Driving west means you'll be heading toward Maharashtra, which is directly west of Telangana.

3. **Estimated Distance**: A distance of 300 km takes you through various terrain and could put you near several towns or cities in Maharashtra.

4. **Potential Destinations**:
   - **Aurangabad**: A significant city known for its historical sites, including the Ajanta and Ellora Caves. It is about 300 km from Hyderabad.
   - **Nanded**: Another city that is approximately within this distance and is known for its Gurudwara and historical significance in Sikhism.

5. **Route Options**: Depending on the specific route taken (e.g., via NH65), you might pass through smaller towns as well, like Zaheera

In [20]:
print(answer_based_on_map(CLAUDE,"""
If I drive 300km west of Hyderabad, where will I land up?
Think about it step-by-step.
"""))

Looking at the India National Highway Network Map, I'll determine where you would end up if you drive 300km west of Hyderabad.

Step 1: Locate Hyderabad on the map.
I can see Hyderabad in southern India, in the state of Telangana.

Step 2: Determine what's 300km west of Hyderabad.
Moving west from Hyderabad for approximately 300km would take you toward the western part of Maharashtra state. Based on the map scale and the highway network, this would put you somewhere near or in the vicinity of:

- You would likely be close to areas between Solapur and Mumbai in Maharashtra
- You might be in the eastern parts of Maharashtra, possibly near cities like Nanded or Latur
- You could potentially be approaching the western Maharashtra region

Step 3: Conclusion
If you drive 300km west of Hyderabad, you would most likely end up in western Maharashtra, possibly near Solapur or somewhere between Solapur and the Maharashtra-Karnataka border region. The exact location would depend on the specific ro

## Where CoT won't help: non-sequential logic

Another place where CoT doesn't work is if the reasoning is not sequential.
In bridge, the reasoning is actually probabilistic. You can not decide on the most likely distribution and just play for it.
Experts will also consider whether there is a safety play that gives you an extra chance.

In [24]:
print(zero_shot(GEMINI, """
You are a bridge expert. Guide me on how to play the given suit combination. 

Example 1:
Q: Holding AKJx opposite xxx for no losers.
A: You have 7 cards between the two hands, and the opponents have 6 cards including the Queen.
To have no losers, you have to hope that the opponents' cards are split 3-3 and that
the Queen is onside.  So, finesse the Jack and then play from the top.

Example 2:
Q: Holding AKJx opposite 5 small for no losers.
A: You have 9 cards between the two hands, and the opponents have 4 cards including the Queen.
If both opponents follow, then the most likely split is 2-2. Therefore, play for the Queen to drop.

Example 3:
Q: Holding AQxx opposite Txxx for one loser.
A: You have 8 cards between the two hands, and the opponents have 5 cards including the King, Jack, and Ten.
To have only one loser, you have to hope that one of the opponents has Kx or singleton King or Jack.
So, play low to the Ace, to catch the potential singleton. If an honor falls, play low towards the remaining honor.
If no honor falls, play small in both hands to catch the potential doubleton King.

Q: Holding AKJxx opposite four small, for no losers
"""))

Okay, I'm ready to analyze your suit combination. Let's break down your specific situation:

**Q: Holding AKJxx opposite four small, for no losers**

Here's how I would approach this suit:

**Analysis:**

*   **Combined Strength:** You have a very strong holding with AKJxx.
*   **Missing Cards:** The opponents hold the Queen and the Nine, Eight, Seven, Six, Five, Four, Three, Two.
*   **Goal:** You need to make the hand without losing a trick in the suit.
*   **Total Cards in Suit:** 9 in the two hands, so the opponents have 4.

**The Play:**

The key is to determine the best way to extract the Queen from the opponents' hand to make no losers.

**Play a Small Card from Your Hand with the Four Small:**

The best strategy is to lead a small card from your hand and take it with a high card from the four small cards. This will tell us if the Queen is in one of the hands.

If one opponent wins the trick with the Queen, you have no more losers.

If both opponents follow suit with a small car

Hopelessly wrong:  The expert line is to play the Ace, and then the King if the ten doesn’t fall on the right. If the ten falls, then come to hand in another suit and take the finesse.